In [1]:
import numpy as np
from gensim.models.word2vec import Word2Vec
from gensim.corpora.dictionary import Dictionary
from gensim import models
import pandas as pd
import jieba
import logging
from keras import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Bidirectional,LSTM,Dense,Embedding,Dropout,Activation,Softmax
from keras.models import load_model
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
import pymysql
import numpy
from opencc import OpenCC
import jieba.posseg as jp
from sklearn.metrics import confusion_matrix
import seaborn as sns 
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE, SVMSMOTE
from imblearn.combine import SMOTEENN, SMOTETomek

Using TensorFlow backend.


In [ ]:
sql = "select quest_nm_adj, class_nm from textdb.fact_cls_qa_kw where quest_kw is not null" #SQL指令拉資料

In [ ]:
db = pymysql.connect(host='172.16.56.101', port=31996, user='root', passwd='password', db='textdb' )
cursor = db.cursor()
cursor.execute(sql)
kw_results = cursor.fetchall() #從MYSQL拉出資料

In [2]:
csv_data = pd.read_csv('data/fact_cls_qa_kw.csv', encoding='big5') #從CSV拉資料

In [3]:
kw_results = [] #整理CSV資料
for i in range(len(csv_data)):
    kw_results.append([
    list(csv_data['quest_nm_adj'])[i],
    list(csv_data['class_nm'])[i]])

In [4]:
kw_results[0:5] #資料展示前5筆

[['帳單周期看得到嗎?', '帳務查詢'],
 ['我要請你幫我查一隻電話號碼帳單多少錢', '帳務查詢'],
 ['想詢問這期帳單', '帳務查詢'],
 ['我想要查詢我電話目前帳單地址', '帳務查詢'],
 ['我的帳單每個月是多少錢', '帳務查詢']]

In [5]:
import numpy
class_list_orgn = []
for raw in kw_results:
    rsl = raw[1].split(',')
    for rs in rsl:
        class_list_orgn.append(rs)
unique, counts = numpy.unique(class_list_orgn, return_counts=True)
dict(zip(unique, counts)) #統計各class筆數

{'合約查詢': 10, '帳務查詢': 10, '魔速方塊1.0': 10, '魔速方塊2.0': 10}

In [6]:
i = 0
label_dic = {}
for label_raw in unique:
#     print(label_raw[0])
    label_dic[label_raw] = i
    i += 1
class_list = []
for classes in class_list_orgn:
    class_list.append(label_dic[classes])
    
label_dic #把CLASS名稱轉成數字

{'合約查詢': 0, '帳務查詢': 1, '魔速方塊1.0': 2, '魔速方塊2.0': 3}

In [7]:
data_list = []
for raw in kw_results:
    data_list.append(raw[0])
data_list #訓練句子資料

['帳單周期看得到嗎?',
 '我要請你幫我查一隻電話號碼帳單多少錢',
 '想詢問這期帳單',
 '我想要查詢我電話目前帳單地址',
 '我的帳單每個月是多少錢',
 '我想請問一下你們的帳單截止日是幾號?',
 '我辦理NP後還有一筆帳單還未繳',
 '我這次要繳多少錢',
 '請問我的帳單是幾號繳費',
 '我想要查我手機的帳單',
 '我現在的資費方案是?',
 '我的方案是啥',
 '請告訴我當初辦理的專案',
 '目前資費是什麼',
 '我忘了當初申辦的資費了',
 '我想要解約NP去別家',
 '怎麼辦理解約',
 '不想用這個門號了，我要解約',
 '收訊太爛了，我不想用了啦，立刻給我解約',
 '離譜的訊號品質，給我解了吧',
 '魔速方塊1.0和Wi-Fi通話有什麼不同',
 '魔速方塊1.0設備規格支援哪種頻段',
 '如果用戶非亞太本網，魔速方塊1.0也支援嗎',
 '要去哪裡申請魔速方塊1.0呢',
 '如果有企業用戶要申請魔速方塊，請問要去哪邊申辦',
 '亞太用戶都可以申裝廆速方塊1.0嗎',
 '回到家不會安裝魔速方塊怎麼辦',
 '魔速方塊有問題或需要維修怎麼辦',
 '如果用戶搬家，是否需要歸還魔速方塊',
 '我不小心弄壞魔速方塊1.0了，要賠錢嗎',
 '魔速方塊2.0發射功率為多少',
 '魔速方塊2.0在一般情況下耗電率如何',
 '每天、一個月消耗電量以及電費預估?',
 '魔速方塊2.0是否可以增強網路的數據下載與上傳速率',
 '怎麼申請魔速方塊2.0?需要費用嗎?',
 '魔速方塊2.0功能為何',
 '電器產品擺在魔速方塊2.0附近是否會影響訊號轉換後的功率',
 '魔速方塊2.0是否會跟任何WiFi訊號互相干擾',
 '有沒有環境或條件的外在影響，會造成魔速方塊2.0的訊號功效不如預期?',
 '魔速方塊2.0對高樓層有限制嗎']

In [8]:
cc= OpenCC('s2twp')
words_file = './stopwords.txt' #文本檔案位置
stopwords_set = set()                                           #创建set集合
with open(words_file, 'r', encoding = 'utf-8') as f:            #打开文件
    for line in f.readlines():                                  #一行一行读取
        word = line.strip()                                     #去回车
        if len(word) > 0:                                       #有文本，则添加到words_set中
            stopwords_set.add(cc.convert(word))
stopwords_set #停用字詞

{'一',
 '一些',
 '一來',
 '一切',
 '一旦',
 '一起響應環保愛地球',
 '上',
 '下',
 '不',
 '不但',
 '不僅',
 '不僅僅',
 '不光',
 '不只',
 '不單',
 '不外乎',
 '不好意思',
 '不如',
 '不料',
 '不是',
 '不會',
 '不然',
 '不盡',
 '不盡然',
 '不管',
 '不至於',
 '不論',
 '不過',
 '且',
 '並',
 '並不',
 '並且',
 '並非',
 '乃',
 '乃至',
 '之',
 '之所以',
 '也',
 '了',
 '些',
 '亞太電信推出無紙化簡訊及e-mail電子帳單服務',
 '亦',
 '人',
 '人們',
 '什麼',
 '什麼樣',
 '什麼的',
 '今',
 '介於',
 '仍',
 '仍舊',
 '他',
 '他人',
 '他們',
 '以',
 '以上',
 '以來',
 '以便',
 '以免',
 '以及',
 '以為',
 '以至',
 '以致',
 '任何',
 '似的',
 '但',
 '但是',
 '何',
 '何以',
 '何時',
 '何況',
 '何處',
 '作為',
 '你',
 '你們',
 '使',
 '來',
 '來自',
 '來說',
 '例如',
 '依據',
 '依照',
 '便於',
 '個',
 '個人',
 '個別',
 '們',
 '倘若',
 '值此',
 '假如',
 '儘管',
 '儘管如此',
 '先生',
 '光是',
 '兒',
 '全部',
 '全體',
 '兩者',
 '其',
 '其中',
 '其他',
 '其它',
 '其次',
 '其餘',
 '再',
 '再則',
 '再有',
 '再見',
 '凡',
 '凡是',
 '出來',
 '出於',
 '分別',
 '別',
 '別人',
 '別的',
 '別處',
 '別說',
 '到',
 '則',
 '前者',
 '加之',
 '加以',
 '即',
 '即使',
 '即便',
 '卻',
 '去',
 '又',
 '又及',
 '及',
 '及至',
 '反之',
 '反而',
 '受到',
 '另',
 '另一方面',
 '另外',
 '只',
 '只因',
 '只怕',
 '只是',
 '

In [9]:
jieba.load_userdict("./jieba_dict.txt") #讀取自建辭庫

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.095 seconds.
Prefix dict has been built successfully.


In [10]:
# 建立 W2V model
def train_word2vec(sentences,save_path):
    sentences_seg = []
    for i in sentences:
        sen_proc = []
        try:
            for word in jieba.cut(i):
                sen_proc.append(word)
            sentences_seg.append(sen_proc)
        except:
             continue                
    print("start w2v train") 
    model = Word2Vec(sentences_seg,
                size=100,  # 词向量维度
                min_count=5,  # 词频阈值
                window=5)  # 窗口大小    
    model.save(save_path)
    print("w2v train complete")
    return model, sentences_seg 

In [11]:
# train W2V
model, sentences_seg =  train_word2vec(data_list, './LSTM_model/w2v_model/word2vec.model.bin')   

start w2v train
w2v train complete


In [12]:
# W2V編號與權重
def generate_id2wec(word2vec_model):
    gensim_dict = Dictionary()
    gensim_dict.doc2bow(model.wv.vocab.keys(), allow_update=True)
    w2id = {v: k + 1 for k, v in gensim_dict.items()}  # 词语的索引，从1开始编号
    w2vec = {word: model[word] for word in w2id.keys()}  # 词语的词向量
    n_vocabs = len(w2id) + 1
    embedding_weights = np.zeros((n_vocabs, 100))
    for w, index in w2id.items():  # 从索引为1的词语开始，用词向量填充矩阵
        embedding_weights[index, :] = w2vec[w]
    return w2id,embedding_weights
# 句子轉成W2V編號
def text_to_array(w2index, senlist):  # 文本转为索引数字模式
    sentences_array = []
    for sen in senlist:
        new_sen = [ w2index.get(word,0) for word in sen]   # 单词转索引数字
        sentences_array.append(new_sen)
    return np.array(sentences_array)
# 準備訓練資料
def prepare_data(w2id,sentences,labels,max_len=200):
    X_train, X_val, y_train, y_val = train_test_split(sentences,labels, test_size=0.2)
    X_train = text_to_array(w2id, X_train)
    X_val = text_to_array(w2id, X_val)
    X_train = pad_sequences(X_train, maxlen=max_len)
    X_val = pad_sequences(X_val, maxlen=max_len)
    return np.array(X_train), np_utils.to_categorical(y_train) ,np.array(X_val), np_utils.to_categorical(y_val)

In [13]:
# 產生W2V編號與權重
w2id,embedding_weights = generate_id2wec(model)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [14]:
x_train,y_trian, x_val , y_val = prepare_data(w2id,sentences_seg,class_list,200)

In [15]:
# 建立 LSTM model
class Sentiment:
    def __init__(self,w2id,embedding_weights,Embedding_dim,maxlen,labels_category, save_model_path='./LSTM_model/LSTM_model/sentiment.h5'):
        self.Embedding_dim = Embedding_dim
        self.embedding_weights = embedding_weights
        self.vocab = w2id
        self.labels_category = labels_category
        self.maxlen = maxlen
        self.model = self.build_model()
        self.smp = save_model_path
        
    def build_model(self):
        model = Sequential()
        #input dim(140,100)
        model.add(Embedding(output_dim = self.Embedding_dim,
                           input_dim=len(self.vocab)+1,
                           weights=[self.embedding_weights],
                           input_length=self.maxlen))
        model.add(Bidirectional(LSTM(50),merge_mode='concat'))
        model.add(Dropout(0.5))
        model.add(Dense(self.labels_category))
        model.add(Activation('softmax'))
        model.compile(loss='categorical_crossentropy',
                     optimizer='adam', 
                     metrics=['accuracy'])
        model.summary()
        return model
    
    def train(self,X_train, y_train,X_test, y_test,n_epoch=5 ):
        self.model.fit(X_train, y_train, batch_size=32, epochs=n_epoch,
                      validation_data=(X_test, y_test))
        self.model.save(self.smp)   
        
    def predict(self,model_path,new_sen):
        model = self.model
        model.load_weights(model_path)
        new_sen_list = jieba.lcut(new_sen)
        sen2id =[ self.vocab.get(word,0) for word in new_sen_list]
        sen_input = pad_sequences([sen2id], maxlen=self.maxlen)
        res = model.predict(sen_input)[0]
        return np.argmax(res)

In [ ]:
# 訓練 LSTM
# Sentiment(w2id,embedding_weights, 词向量维度, 句子長度, class數目, model儲存位置')
senti = Sentiment(w2id,embedding_weights,100,200,4, './LSTM_model/LSTM_model/sentiment.h5')
# senti.train(x_train,y_trian, x_val ,y_val, Epoch次數)
senti.train(x_train,y_trian, x_val ,y_val,5)

In [ ]:
# 讀取儲存LSTM model
model = load_model('./LSTM_model/LSTM_model/sentiment.h5')

In [ ]:
# 測試精準度與混淆矩陣
prc = []
for prw in data_list:
    new_sen_list = jieba.lcut(prw)
    sen2id = [w2id.get(word,0) for word in new_sen_list]
    sen_input = pad_sequences([sen2id], maxlen=200)
    res = model.predict(sen_input)[0]
    prc.append(np.argmax(res))
ps = 0
for pi in range(len(class_list)):
    if class_list[pi] == prc[pi]:
        ps += 1
prs = ps / len(class_list)
prs
confusion = confusion_matrix(class_list, prc, labels=[num for num in range(len(label_dic))]) #與實際答案做混淆矩陣
plt.figure()
print(label_dic)
print('acc: %.2f' % (prs*100) + '%')
sns.heatmap(confusion,annot=True, xticklabels=[num for num in range(len(label_dic))], yticklabels=[num for num in range(len(label_dic))], annot_kws={"size": 20}) #畫出熱度圖
plt.show(sns)

In [ ]:
# SMOTE資料增強

# SMOTE
oversampler=SMOTE(random_state=45)
x_resampled, y_resampled = oversampler.fit_sample(x_train, y_trian)
senti = Sentiment(w2id,embedding_weights,100,200,4, './LSTM_model/LSTM_model/sentiment_s1.h5')
senti.train(x_resampled,y_resampled, x_val ,y_val,5)
# SMOTE kind=borderline-1
oversampler=BorderlineSMOTE(random_state=45, kind='borderline-1')
x_resampled, y_resampled = oversampler.fit_sample(x_train, y_trian)
senti = Sentiment(w2id,embedding_weights,100,200,4, './LSTM_model/LSTM_model/sentiment_s2.h5')
senti.train(x_resampled,y_resampled, x_val ,y_val,5)
# SMOTE kind=borderline-2
oversampler=BorderlineSMOTE(random_state=45, kind='borderline-2')
x_resampled, y_resampled = oversampler.fit_sample(x_train, y_trian)
senti = Sentiment(w2id,embedding_weights,100,200,4, './LSTM_model/LSTM_model/sentiment_s3.h5')
senti.train(x_resampled,y_resampled, x_val ,y_val,5)
# SMOTEENN
oversampler=SMOTEENN(random_state=45)
x_resampled, y_resampled = oversampler.fit_sample(x_train, y_trian)
senti = Sentiment(w2id,embedding_weights,100,200,4, './LSTM_model/LSTM_model/sentiment_s4.h5')
senti.train(x_resampled,y_resampled, x_val ,y_val,5)
# SMOTETomek
oversampler=SMOTETomek(random_state=45)
x_resampled, y_resampled = oversampler.fit_sample(x_train, y_trian)
senti = Sentiment(w2id,embedding_weights,100,200,4, './LSTM_model/LSTM_model/sentiment_s5.h5')
senti.train(x_resampled,y_resampled, x_val ,y_val,5)
# ADASYN
try:
    oversampler=ADASYN(random_state=45)
    x_resampled, y_resampled = oversampler.fit_sample(x_train, y_trian)
    senti = Sentiment(w2id,embedding_weights,100,200,4, './LSTM_model/LSTM_model/sentiment_s6.h5')
    senti.train(x_resampled,y_resampled, x_val ,y_val,5)
except ValueError:
    print('ADASYN: No samples will be generated with the provided ratio settings.')
# SVMSMOTE
oversampler=SVMSMOTE(random_state=45)
x_resampled, y_resampled = oversampler.fit_sample(x_train, y_trian)
senti = Sentiment(w2id,embedding_weights,100,200,4, './LSTM_model/LSTM_model/sentiment_s7.h5')
senti.train(x_resampled,y_resampled, x_val ,y_val,5)

In [ ]:
#讀取SMOTE資料增強後儲存LSTM model
model1 = load_model('./LSTM_model/LSTM_model/sentiment_s1.h5')
model2 = load_model('./LSTM_model/LSTM_model/sentiment_s2.h5')
model3 = load_model('./LSTM_model/LSTM_model/sentiment_s3.h5')
model4 = load_model('./LSTM_model/LSTM_model/sentiment_s4.h5')
model5 = load_model('./LSTM_model/LSTM_model/sentiment_s5.h5')
# model6 = load_model('./LSTM_model/LSTM_model/sentiment_s6.h5')
model7 = load_model('./LSTM_model/LSTM_model/sentiment_s7.h5')

In [ ]:
#測試SMOTE資料增強後精準度與混淆矩陣

print(label_dic)
plt.figure(figsize=(15,20))
plt.subplot(4,2,1)
plt.title('origin predict: ' + str(round(prs*100, 2)) + ' %')
sns.heatmap(confusion,annot=True, xticklabels=[num for num in range(len(label_dic))], yticklabels=[num for num in range(len(label_dic))], annot_kws={"size": 20}) #畫出熱度圖

# print('==============SMOTE=====================================')
prc = []
for prw in data_list:
    new_sen_list = jieba.lcut(prw)
    sen2id = [w2id.get(word,0) for word in new_sen_list]
    sen_input = pad_sequences([sen2id], maxlen=200)
    res = model1.predict(sen_input)[0]
    prc.append(np.argmax(res))
ps = 0
for pi in range(len(class_list)):
    if class_list[pi] == prc[pi]:
        ps += 1
prs = ps / len(class_list)
prs
confusion = confusion_matrix(class_list, prc, labels=[num for num in range(len(label_dic))]) #與實際答案做混淆矩陣
plt.subplot(4,2,2)
plt.title('SMOTE predict: ' + str(round(prs*100, 2)) + ' %')
sns.heatmap(confusion,annot=True, xticklabels=[num for num in range(len(label_dic))], yticklabels=[num for num in range(len(label_dic))], annot_kws={"size": 20}) #畫出熱度圖

# print('==============SMOTE kind=borderline-1=====================================')
prc = []
for prw in data_list:
    new_sen_list = jieba.lcut(prw)
    sen2id = [w2id.get(word,0) for word in new_sen_list]
    sen_input = pad_sequences([sen2id], maxlen=200)
    res = model2.predict(sen_input)[0]
    prc.append(np.argmax(res))
ps = 0
for pi in range(len(class_list)):
    if class_list[pi] == prc[pi]:
        ps += 1
prs = ps / len(class_list)
prs
confusion = confusion_matrix(class_list, prc, labels=[num for num in range(len(label_dic))]) #與實際答案做混淆矩陣
plt.subplot(4,2,3)
plt.title('SMOTE kind=borderline-1 predict: ' + str(round(prs*100, 2)) + ' %')
sns.heatmap(confusion,annot=True, xticklabels=[num for num in range(len(label_dic))], yticklabels=[num for num in range(len(label_dic))], annot_kws={"size": 20}) #畫出熱度圖

# print('==============SMOTE kind=borderline-2=====================================')
prc = []
for prw in data_list:
    new_sen_list = jieba.lcut(prw)
    sen2id = [w2id.get(word,0) for word in new_sen_list]
    sen_input = pad_sequences([sen2id], maxlen=200)
    res = model3.predict(sen_input)[0]
    prc.append(np.argmax(res))
ps = 0
for pi in range(len(class_list)):
    if class_list[pi] == prc[pi]:
        ps += 1
prs = ps / len(class_list)
prs
confusion = confusion_matrix(class_list, prc, labels=[num for num in range(len(label_dic))]) #與實際答案做混淆矩陣
plt.subplot(4,2,4)
plt.title('SMOTE kind=borderline-2 predict: ' + str(round(prs*100, 2)) + ' %')
sns.heatmap(confusion,annot=True, xticklabels=[num for num in range(len(label_dic))], yticklabels=[num for num in range(len(label_dic))], annot_kws={"size": 20}) #畫出熱度圖

# print('==============SMOTEENN=====================================')
prc = []
for prw in data_list:
    new_sen_list = jieba.lcut(prw)
    sen2id = [w2id.get(word,0) for word in new_sen_list]
    sen_input = pad_sequences([sen2id], maxlen=200)
    res = model4.predict(sen_input)[0]
    prc.append(np.argmax(res))
ps = 0
for pi in range(len(class_list)):
    if class_list[pi] == prc[pi]:
        ps += 1
prs = ps / len(class_list)
prs
confusion = confusion_matrix(class_list, prc, labels=[num for num in range(len(label_dic))]) #與實際答案做混淆矩陣
plt.subplot(4,2,5)
plt.title('SMOTEENN predict: ' + str(round(prs*100, 2)) + ' %')
sns.heatmap(confusion,annot=True, xticklabels=[num for num in range(len(label_dic))], yticklabels=[num for num in range(len(label_dic))], annot_kws={"size": 20}) #畫出熱度圖

# print('==============SMOTETomek=====================================')
prc = []
for prw in data_list:
    new_sen_list = jieba.lcut(prw)
    sen2id = [w2id.get(word,0) for word in new_sen_list]
    sen_input = pad_sequences([sen2id], maxlen=200)
    res = model5.predict(sen_input)[0]
    prc.append(np.argmax(res))
ps = 0
for pi in range(len(class_list)):
    if class_list[pi] == prc[pi]:
        ps += 1
prs = ps / len(class_list)
prs
confusion = confusion_matrix(class_list, prc, labels=[num for num in range(len(label_dic))]) #與實際答案做混淆矩陣
plt.subplot(4,2,6)
plt.title('SMOTETomek predict: ' + str(round(prs*100, 2)) + ' %')
sns.heatmap(confusion,annot=True, xticklabels=[num for num in range(len(label_dic))], yticklabels=[num for num in range(len(label_dic))], annot_kws={"size": 20}) #畫出熱度圖

# print('==============ADASYN=====================================')
# prc = []
# for prw in data_list:
#     new_sen_list = jieba.lcut(prw)
#     sen2id = [w2id.get(word,0) for word in new_sen_list]
#     sen_input = pad_sequences([sen2id], maxlen=200)
#     res = model6.predict(sen_input)[0]
#     prc.append(np.argmax(res))
# ps = 0
# for pi in range(len(class_list)):
#     if class_list[pi] == prc[pi]:
#         ps += 1
# prs = ps / len(class_list)
# prs
# confusion = confusion_matrix(class_list, prc, labels=[num for num in range(len(label_dic))]) #與實際答案做混淆矩陣
# plt.subplot(4,2,7)
# plt.title('ADASYN predict: ' + str(round(prs*100, 2)) + ' %')
# sns.heatmap(confusion,annot=True, xticklabels=[num for num in range(len(label_dic))], yticklabels=[num for num in range(len(label_dic))], annot_kws={"size": 20}) #畫出熱度圖

# print('==============SVMSMOTE=====================================')
prc = []
for prw in data_list:
    new_sen_list = jieba.lcut(prw)
    sen2id = [w2id.get(word,0) for word in new_sen_list]
    sen_input = pad_sequences([sen2id], maxlen=200)
    res = model7.predict(sen_input)[0]
    prc.append(np.argmax(res))
ps = 0
for pi in range(len(class_list)):
    if class_list[pi] == prc[pi]:
        ps += 1
prs = ps / len(class_list)
prs
confusion = confusion_matrix(class_list, prc, labels=[num for num in range(len(label_dic))]) #與實際答案做混淆矩陣
plt.subplot(4,2,8)
plt.title('SVMSMOTE predict: ' + str(round(prs*100, 2)) + ' %')
sns.heatmap(confusion,annot=True, xticklabels=[num for num in range(len(label_dic))], yticklabels=[num for num in range(len(label_dic))], annot_kws={"size": 20}) #畫出熱度圖


In [ ]:
# LSTM model 句子測試

new_sen = '解約金是多少'

new_sen_list = jieba.lcut(new_sen)
sen2id = [w2id.get(word,0) for word in new_sen_list]
sen_input = pad_sequences([sen2id], maxlen=200)
res = model.predict(sen_input)[0]
print('origin: ' + list(label_dic.keys())[list(label_dic.values()).index(np.argmax(res))])
res = model1.predict(sen_input)[0]
print('SMOTE: ' + list(label_dic.keys())[list(label_dic.values()).index(np.argmax(res))])
res = model2.predict(sen_input)[0]
print('SMOTE kind=borderline-1: ' + list(label_dic.keys())[list(label_dic.values()).index(np.argmax(res))])
res = model3.predict(sen_input)[0]
print('SMOTE kind=borderline-2: ' + list(label_dic.keys())[list(label_dic.values()).index(np.argmax(res))])
res = model4.predict(sen_input)[0]
print('SMOTEENN: ' + list(label_dic.keys())[list(label_dic.values()).index(np.argmax(res))])
res = model5.predict(sen_input)[0]
print('SMOTETomek: ' + list(label_dic.keys())[list(label_dic.values()).index(np.argmax(res))])
# res = model6.predict(sen_input)[0]
# print('ADASYN: ' +  list(label_dic.keys())[list(label_dic.values()).index(np.argmax(res))])
res = model7.predict(sen_input)[0]
print('SVMSMOTE: ' + list(label_dic.keys())[list(label_dic.values()).index(np.argmax(res))])

In [ ]:
'sentiment.h5'[:-3]